# 房贷还款计算器

用于对房贷还款计划做决策。对比还款前后总支付金额。同时对比将资金用于其他投资产生的收益情况。

投资有风险，决策须谨慎，投资收益的计算结果是假设无风险且收益率可保证在设置的最低水平，且可以至少持续到贷款结束的时间。

## 基础知识

### 机会成本

机会成本是指在做出某种选择后，次优选项所带来的损失。

### 复利

复利的计算方法是：

$$
F = P \times (1 + r)^n
$$

其中，$F$ 是最终金额，$P$ 是本金，$r$ 是利率，$n$ 是计息期数。如果 $r$ 是年利率，则期数$n$是计息年数。

### 计算结果解释

我们发现计算结果显示，投资的收益率远低于贷款的利率，但是最终的收益却比贷款的利息高。这是因为贷款的本金随着月供而逐渐减少，而投资的本金持续不变。因此，虽然投资的收益率较低，但是最终的收益却更高。

## 贷款信息设置

In [66]:
import pandas as pd
from IPython.display import Markdown


# 贷款剩余本金
principal_before = 100000
# 年利率
rate_year = 3.9/100
# 贷款剩余期数
term = 240

# 提前还款的本金
return_amount = 10000
# 提前还款支付的手续费、利息
return_intrest = 0

# 机会成本计算(年利率)
invest_intrest_year = 3.9/100


# 月利率计算
rate_month = rate_year/12
invest_intrest_month = invest_intrest_year/12

## 等额本金

In [67]:
decrease_before = principal_before / term
decrease_after = (principal_before - return_amount) / term
remain_before = principal_before
remain_after = principal_before - return_amount
payments_data = []

total_payments_before = (
    principal_before * rate_month + decrease_before * rate_month
) * term / 2 + principal_before

total_payments_after = (
    (remain_after * rate_month + decrease_after * rate_month) * term / 2
    + remain_after
    + return_amount
    + return_intrest
)

investment_balance = 0
total_reduced = 0
for i in range(term):
    interest_before = remain_before * rate_month
    remain_before = remain_before - decrease_before

    interest_after = remain_after * rate_month
    remain_after = remain_after - decrease_after
    
    reduce_monthly = decrease_before+interest_before-(decrease_after+interest_after)
    investment_balance = investment_balance  * (1+invest_intrest_month) + reduce_monthly
    total_reduced = total_reduced + reduce_monthly

    payments_data.append(
        {
            "月": i + 1,
            "本金(前)": decrease_before, 
            "利息(前)": interest_before, 
            "月供(前)": decrease_before + interest_before,
            "剩余本金(前)": remain_before,
            "本金(后)": decrease_after,
            "利息(后)": interest_after,
            "月供(后)": decrease_after + interest_after,
            "剩余本金(后)": remain_after,
            "月供减少": reduce_monthly,
            "定投余额": investment_balance,
        }
    )
display(
    Markdown(
        ('<p style="font-size: 16px">当前剩余贷款本金是{}， 年化利率是{}%，{}个月等额本金法支付本息之和是{}。</p>'
        +'<p style="font-size: 16px">当前提前还款本金{}，还款手续费/利息{}。</p>'
        # +'<p style="font-size: 16px">如果提前还款支付本息之和是{}。共计减少支付{}。</p>'
        +'<p style="font-size: 16px">如果提前还款，未来支付本息之和是{}，减少支付利息{}。将节省资金定投，{}个月后可获得本息之和{}(利息{}）。减少利息+获得利息={}</p>'
        +'<p style="font-size: 16px">如果将{}用于年化 {}% 的无风险投资。{}个月最终可获得的收益是{}（复利法）。</p> ').format(
            round(principal_before, 2),
            round(rate_year * 100, 2),
            term,
            round(total_payments_before, 2),
            
            round(return_amount, 2),
            round(return_intrest, 2),
            
            round(total_payments_after, 2),
            round(total_payments_before - total_payments_after, 2),
            term,
            round(investment_balance, 2),
            round(investment_balance - total_reduced, 2),
            round(total_payments_before - total_payments_after+(investment_balance - total_reduced),2),
            
            round(return_amount + return_intrest, 2),
            round(invest_intrest_year * 100, 2),
            term,
            round(
                (return_amount + return_intrest) * (1 + invest_intrest_month) ** term
                - (return_amount + return_intrest),
                2,
            ),
        )
    )
)

df = pd.DataFrame(payments_data)
display(
    df.style.format(
        {
            "本金(前)": "¥{:,.2f}",
            "利息(前)": "¥{:,.2f}",
            "月供(前)": "¥{:,.2f}",
            "剩余本金(前)": "¥{:,.2f}",
            "本金(后)": "¥{:,.2f}",
            "利息(后)": "¥{:,.2f}",
            "月供(后)": "¥{:,.2f}",
            "剩余本金(后)": "¥{:,.2f}",
            "月供减少": "¥{:,.2f}",
            "定投余额": "¥{:,.2f}",
        }
    )
)

<p style="font-size: 16px">当前剩余贷款本金是100000， 年化利率是3.9%，240个月等额本金法支付本息之和是139162.5。</p><p style="font-size: 16px">当前提前还款本金10000，还款手续费/利息0。</p><p style="font-size: 16px">如果提前还款，未来支付本息之和是135246.25，减少支付利息3916.25。将节省资金定投，240个月后可获得本息之和21787.15(利息7870.9）。减少利息+获得利息=11787.15</p><p style="font-size: 16px">如果将10000用于年化 3.9% 的无风险投资。240个月最终可获得的收益是11787.15（复利法）。</p> 

,月,本金(前),利息(前),月供(前),剩余本金(前),本金(后),利息(后),月供(后),剩余本金(后),月供减少,定投余额
0,1,¥416.67,¥325.00,¥741.67,"¥99,583.33",¥375.00,¥292.50,¥667.50,"¥89,625.00",¥74.17,¥74.17
1,2,¥416.67,¥323.65,¥740.31,"¥99,166.67",¥375.00,¥291.28,¥666.28,"¥89,250.00",¥74.03,¥148.44
2,3,¥416.67,¥322.29,¥738.96,"¥98,750.00",¥375.00,¥290.06,¥665.06,"¥88,875.00",¥73.90,¥222.82
3,4,¥416.67,¥320.94,¥737.60,"¥98,333.33",¥375.00,¥288.84,¥663.84,"¥88,500.00",¥73.76,¥297.30
4,5,¥416.67,¥319.58,¥736.25,"¥97,916.67",¥375.00,¥287.62,¥662.62,"¥88,125.00",¥73.62,¥371.89
5,6,¥416.67,¥318.23,¥734.90,"¥97,500.00",¥375.00,¥286.41,¥661.41,"¥87,750.00",¥73.49,¥446.59
6,7,¥416.67,¥316.87,¥733.54,"¥97,083.33",¥375.00,¥285.19,¥660.19,"¥87,375.00",¥73.35,¥521.40
7,8,¥416.67,¥315.52,¥732.19,"¥96,666.67",¥375.00,¥283.97,¥658.97,"¥87,000.00",¥73.22,¥596.31
8,9,¥416.67,¥314.17,¥730.83,"¥96,250.00",¥375.00,¥282.75,¥657.75,"¥86,625.00",¥73.08,¥671.33
9,10,¥416.67,¥312.81,¥729.48,"¥95,833.33",¥375.00,¥281.53,¥656.53,"¥86,250.00",¥72.95,¥746.46


## 等额本息

In [68]:
monthly_before = (
    principal_before
    * rate_month * (1 + rate_month) ** term
    / ((1 + rate_month) ** term - 1)
)

monthly_after = (
    (principal_before - return_amount)
    * rate_month
    * (1 + rate_month) ** term
    / ((1 + rate_month) ** term - 1)
)

monthly_investment_final = (monthly_before-monthly_after) * ((1 + invest_intrest_month) ** term - 1) / invest_intrest_month

total_payments_before = monthly_before * term
total_payments_after = monthly_after * term + return_amount + return_intrest

display(
    Markdown(
        ('<p style="font-size: 16px">当前剩余贷款本金是{}， 年化利率是{}%，{}个月等额本息法支付本息之和是{}。</p>'
        +'<p style="font-size: 16px">当前提前还款本金{}，还款手续费（或利息）{}。</p>'
        +'<p style="font-size: 16px">如果提前还款，未来支付本息之和是{}，减少支付利息{}。每月节省月供{}，将节省资金定投，{}个月后可获得本息之和{}(利息{})。减少利息+获得利息={}。</p>'
        +'<p style="font-size: 16px">如果将{}用于年化 {}% 的无风险投资。{}个月最终可获得的收益是{}（复利法）。</p> ').format(
            round(principal_before, 2),
            round(rate_year * 100, 2),
            term,
            round(total_payments_before, 2),
            
            round(return_amount, 2),
            round(return_intrest, 2),
            
            round(total_payments_after, 2),
            round(total_payments_before - total_payments_after, 2),
            round(monthly_before - monthly_after, 2),
            term,
            round(monthly_investment_final, 2),
            round(
                monthly_investment_final - (monthly_before - monthly_after) * term, 
                2,
            ),
            round(monthly_investment_final - (monthly_before - monthly_after) * term + total_payments_before - total_payments_after, 2),
            round(return_amount + return_intrest, 2),
            round(invest_intrest_year * 100, 2),
            term,
            round(
                (return_amount + return_intrest) * (1 + invest_intrest_month) ** term
                - (return_amount + return_intrest),
                2,
            ),
        )
    )
)

remaining_before = principal_before
remaining_after = principal_before - return_amount
payments_data = []
investment_balance = 0
for month in range(term):
    interest_before = remaining_before * rate_month
    decrease_before = monthly_before - interest_before
    remaining_before -= decrease_before

    interest_after = remaining_after * rate_month
    decrease_after = monthly_after - interest_after
    remaining_after -= decrease_after

    reduce_monthly = decrease_before+interest_before-(decrease_after+interest_after)
    # investment_balance = (investment_balance + reduce_monthly)  * (1+invest_intrest_month) 
    investment_balance = investment_balance  * (1+invest_intrest_month) + reduce_monthly

    payments_data.append(
        {
            "月": month + 1,
            "利息(前)": interest_before,
            "本金(前)": decrease_before,
            "月供(前)": interest_before + decrease_before,
            "剩余本金(前)": remaining_before,
            "利息(后)": interest_after,
            "本金(后)": decrease_after,
            "月供(后)": interest_after + decrease_after,
            "剩余本金(后)": remaining_after,
            "月供减少": reduce_monthly,
            "定投余额": investment_balance,
        }
    )

df = pd.DataFrame(payments_data)
display(
    df.style.format(
        {
            "本金(前)": "¥{:,.2f}",
            "利息(前)": "¥{:,.2f}",
            "月供(前)": "¥{:,.2f}",
            "剩余本金(前)": "¥{:,.2f}",
            "本金(后)": "¥{:,.2f}",
            "利息(后)": "¥{:,.2f}",
            "月供(后)": "¥{:,.2f}",
            "剩余本金(后)": "¥{:,.2f}",
            "月供减少": "¥{:,.2f}",
            "定投余额": "¥{:,.2f}",
        }
    )
)

<p style="font-size: 16px">当前剩余贷款本金是100000， 年化利率是3.9%，240个月等额本息法支付本息之和是144173.76。</p><p style="font-size: 16px">当前提前还款本金10000，还款手续费（或利息）0。</p><p style="font-size: 16px">如果提前还款，未来支付本息之和是139756.38，减少支付利息4417.38。每月节省月供60.07，将节省资金定投，240个月后可获得本息之和21787.15(利息7369.77)。减少利息+获得利息=11787.15。</p><p style="font-size: 16px">如果将10000用于年化 3.9% 的无风险投资。240个月最终可获得的收益是11787.15（复利法）。</p> 

,月,利息(前),本金(前),月供(前),剩余本金(前),利息(后),本金(后),月供(后),剩余本金(后),月供减少,定投余额
0,1,¥325.00,¥275.72,¥600.72,"¥99,724.28",¥292.50,¥248.15,¥540.65,"¥89,751.85",¥60.07,¥60.07
1,2,¥324.10,¥276.62,¥600.72,"¥99,447.66",¥291.69,¥248.96,¥540.65,"¥89,502.89",¥60.07,¥120.34
2,3,¥323.20,¥277.52,¥600.72,"¥99,170.14",¥290.88,¥249.77,¥540.65,"¥89,253.12",¥60.07,¥180.80
3,4,¥322.30,¥278.42,¥600.72,"¥98,891.72",¥290.07,¥250.58,¥540.65,"¥89,002.54",¥60.07,¥241.46
4,5,¥321.40,¥279.33,¥600.72,"¥98,612.39",¥289.26,¥251.39,¥540.65,"¥88,751.15",¥60.07,¥302.32
5,6,¥320.49,¥280.23,¥600.72,"¥98,332.16",¥288.44,¥252.21,¥540.65,"¥88,498.94",¥60.07,¥363.38
6,7,¥319.58,¥281.14,¥600.72,"¥98,051.01",¥287.62,¥253.03,¥540.65,"¥88,245.91",¥60.07,¥424.63
7,8,¥318.67,¥282.06,¥600.72,"¥97,768.95",¥286.80,¥253.85,¥540.65,"¥87,992.06",¥60.07,¥486.08
8,9,¥317.75,¥282.97,¥600.72,"¥97,485.98",¥285.97,¥254.68,¥540.65,"¥87,737.38",¥60.07,¥547.73
9,10,¥316.83,¥283.89,¥600.72,"¥97,202.08",¥285.15,¥255.51,¥540.65,"¥87,481.88",¥60.07,¥609.59
